##Reduce / transform Census Tract data
[Data](https://docs.google.com/file/d/0B1aa6nX82m2WY2RBNERsd0VfN1k/edit)  
[US Census Documentation](http://www.census.gov/acs/www/data_documentation/documentation_main/)

In [56]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [57]:
import pandas as pd
from pprint import pprint

In [58]:
path_base = ("/Users/brian/Google Drive/")
path_specfic = "SPHIP/2009-2013 ACS Demographic Data/Census Tract Data Files and Documentation/"
years = ['2', '3', '4', '5']
year = years[0] # Later this will 'for year in years:'

In [59]:
file_name = 'ACS_13_5YR_DP0'+year+'.txt'
with open(path_base+path_specfic+file_name) as f:
    text_description = f.read()
print(text_description[:1000])
print('...')

DP05
ACS DEMOGRAPHIC AND HOUSING ESTIMATES

Although the American Community Survey (ACS) produces population, demographic and housing unit estimates, it is the Census Bureau's Population Estimates Program that produces and disseminates the official estimates of the population for the nation, states, counties, cities and towns and estimates of housing units for states and counties.


Supporting documentation on code lists, subject definitions, data accuracy, and statistical testing can be found on the American Community Survey website in the Data and Documentation section.

Sample size and data quality measures (including coverage rates, allocation rates, and response rates) can be found on the American Community Survey website in the Methodology section.


Source:  U.S. Census Bureau, 2009-2013 5-Year American Community Survey


Explanation of Symbols:An '**' entry in the margin of error column indicates that either no sample observations or too few sample observations were available t

In [60]:
# Load meta data
file_name = "ACS_13_5YR_DP0"+year+"_metadata.csv"
path_name = (path_base+path_specfic+file_name)
df_meta = pd.read_csv(path_name,
                      header=None)
df_meta.columns = ['data_column_name', 'description']
df_meta.head()

,data_column_name,description
0,GEO.id,Id
1,GEO.id2,Id2
2,GEO.display-label,Geography
3,HC01_VC03,Estimate; SEX AND AGE - Total population
4,HC02_VC03,Margin of Error; SEX AND AGE - Total population


In [70]:
# Load data
file_name = "ACS_13_5YR_DP0"+year+"_with_ann.csv"
path_name = (path_base+path_specfic+file_name)
df_data_with_ann = pd.read_csv(path_name)

# Create data dictionary
df_data_with_ann[:1] # 1st row is annotations / meta data
data_dictionary = df_data_with_ann[:1].T.to_dict()[0]

# Drop annotations/meta data from dataframe
df_data = df_data_with_ann.drop(df_data_with_ann.head(n=1).index)

In [63]:
# Compare data dictionary to meta-data-frame (they should be the same)
df_meta.set_index('data_column_name')['description'].to_dict() == data_dictionary

True

In [71]:
# Improve column naming

# Data dataframe
# Lowercase
df_data.columns = [c.lower() for c in df_data.columns] 
# Rename
df_data = df_data.rename(columns={'geo.id': 'geo_id',
                                  'geo.id2': 'geo_id2',
                                  'geo.display-label': 'geo_display_label'
                            })

# Meta dataframe
# Lowercase
df_meta.data_column_name = df_meta.data_column_name.apply(lambda x: x.lower())
# Rename
df_meta.ix[0,'data_column_name'] = 'geo_id'
df_meta.ix[1,'data_column_name'] = 'geo_id2'
df_meta.ix[2,'data_column_name'] = 'geo_display_label'
df_meta.head(n=4)

# Data dictionary
# Lowercase
data_dictionary = {k.lower(): v for k, v in data_dictionary.items()}
# Rename
data_dictionary['geo_display_label'] = data_dictionary.pop('geo.display-label')
data_dictionary['geo_id'] = data_dictionary.pop('geo.id')
data_dictionary['geo_display_label'] = data_dictionary.pop('geo.id2')

In [86]:
print('data dictionary')
pprint(data_dictionary)
df_meta.head(n=2)

data dictionary
{'geo_display_label': 'Id2',
 'geo_id': 'Id',
 'hc01_vc03': 'Estimate; SEX AND AGE - Total population',
 'hc01_vc04': 'Estimate; SEX AND AGE - Total population - Male',
 'hc01_vc05': 'Estimate; SEX AND AGE - Total population - Female',
 'hc01_vc08': 'Estimate; SEX AND AGE - Under 5 years',
 'hc01_vc09': 'Estimate; SEX AND AGE - 5 to 9 years',
 'hc01_vc10': 'Estimate; SEX AND AGE - 10 to 14 years',
 'hc01_vc100': 'Estimate; HISPANIC OR LATINO AND RACE - Total population - '
               'Not Hispanic or Latino - Two or more races',
 'hc01_vc101': 'Estimate; HISPANIC OR LATINO AND RACE - Total population - '
               'Not Hispanic or Latino - Two or more races - Two races '
               'including Some other race',
 'hc01_vc102': 'Estimate; HISPANIC OR LATINO AND RACE - Total population - '
               'Not Hispanic or Latino - Two or more races - Two races '
               'excluding Some other race, and Three or more races',
 'hc01_vc104': 'Estimate; HISPAN

,data_column_name,description
0,geo_id,Id
1,geo_id2,Id2


In [76]:
print("Data:")
df_data.head(n=2)

Data:


,geo_id,geo_id2,geo_display_label,hc01_vc03,hc02_vc03,hc03_vc03,hc04_vc03,hc01_vc04,hc02_vc04,hc03_vc04,...,hc03_vc101,hc04_vc101,hc01_vc102,hc02_vc102,hc03_vc102,hc04_vc102,hc01_vc104,hc02_vc104,hc03_vc104,hc04_vc104
1,1400000US06075010100,06075010100,"Census Tract 101, San Francisco County, Califo...",3741,358,3741,(X),2051,343,54.8,...,1.4,2.1,192,177,5.1,4.7,2306,84,(X),(X)
2,1400000US06075010200,06075010200,"Census Tract 102, San Francisco County, Califo...",4028,325,4028,(X),2017,213,50.1,...,0.0,0.9,137,93,3.4,2.3,2948,108,(X),(X)


Ideas and work is tracked on [Hackpad](https://datakindsfbayarea.hackpad.com/SF-Health-Improvement-Partnership-SFHIP-IdGfO4Yn60V)

In [67]:
# Visualize each column (e.g., empricial distrbutions)

In [87]:
# Calculate summary stats (e.g., range, min, max)
# df_data.geo_id2.nunique()

# df_data.hc01_vc29.describe().T

In [88]:
# Filter / reduce data
# Define these groups:
#     ACS_13_5YR_DP02_metadata
#     Total households
#     Families with children 
#     Under 12
#     12 to 18
#     19 to 64
#     65 and over
#     Single Parents
#     Under 12
#     12 to 18
#     19 to 64
#     65 and over
#     Average family size
#     Estimate; HOUSEHOLDS BY TYPE - Average family size
#     Educational Attainment
#     No Educational Attainment
#     High School
#     Collect
#     Master and Beyond
#     Disability
#     Percent; DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION - 18 to 64 years
#     Percent; PLACE OF BIRTH - Total population - Native

In [95]:
#Estimate; HOUSEHOLDS BY TYPE - Total households
data_dictionary['hc05_vc03']

KeyError: 'hc05_vc03'

In [108]:
# Find columns that have name
phrase = 'children'
[value for value in data_dictionary.values()]# if value.find(phrase) > 0]

['Margin of Error; RACE - One race - Native Hawaiian and Other Pacific Islander',
 'Estimate; SEX AND AGE - Under 5 years',
 'Margin of Error; RACE - One race - White',
 'Percent; RACE - One race - Asian - Filipino',
 'Estimate; SEX AND AGE - 21 years and over',
 'Estimate; SEX AND AGE - Median age (years)',
 'Estimate; RACE - Two or more races - White and Asian',
 'Margin of Error; SEX AND AGE - 65 years and over - Female',
 'Margin of Error; SEX AND AGE - Under 5 years',
 'Margin of Error; RACE - One race - Native Hawaiian and Other Pacific Islander - Native Hawaiian',
 'Margin of Error; SEX AND AGE - 21 years and over',
 'Estimate; SEX AND AGE - 5 to 9 years',
 'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - White alone',
 'Percent Margin of Error; RACE - One race',
 'Estimate; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race) - Cuban',
 'Margin of Error; RACE - Race alone or in combination with one or more other ra

In [101]:
value = 'Estimate; HOUSEHOLDS BY TYPE - Total households'

In [110]:
df_meta.description.

0                                                     Id
1                                                    Id2
2                                              Geography
3               Estimate; SEX AND AGE - Total population
4        Margin of Error; SEX AND AGE - Total population
5                Percent; SEX AND AGE - Total population
6      Percent Margin of Error; SEX AND AGE - Total p...
7        Estimate; SEX AND AGE - Total population - Male
8      Margin of Error; SEX AND AGE - Total populatio...
9         Percent; SEX AND AGE - Total population - Male
10     Percent Margin of Error; SEX AND AGE - Total p...
11     Estimate; SEX AND AGE - Total population - Female
12     Margin of Error; SEX AND AGE - Total populatio...
13      Percent; SEX AND AGE - Total population - Female
14     Percent Margin of Error; SEX AND AGE - Total p...
15                 Estimate; SEX AND AGE - Under 5 years
16          Margin of Error; SEX AND AGE - Under 5 years
17                  Percent; SE